In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, AveragePooling2D, 
                                     Flatten, Dense, Dropout, BatchNormalization,
                                     GlobalAveragePooling2D)
from tensorflow.keras.callbacks import (ModelCheckpoint, EarlyStopping, 
                                        ReduceLROnPlateau, LearningRateScheduler)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import os
import random
from PIL import Image
import time
import warnings
warnings.filterwarnings('ignore')
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print(f"Versión de TensorFlow: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

In [ ]:
def load_and_display_sample_images():
    """Cargar y mostrar imágenes de muestra de ambas carpetas"""
    folder1 = './skin-cancer-mnist-ham10000/HAM10000_images_part_1'
    folder2 = './skin-cancer-mnist-ham10000/HAM10000_images_part_2'
    
    if os.path.exists(folder1) and os.path.exists(folder2):
        images1 = [os.path.join(folder1, f) for f in os.listdir(folder1) if f.endswith('.jpg')]
        images2 = [os.path.join(folder2, f) for f in os.listdir(folder2) if f.endswith('.jpg')]
        
        random_images1 = random.sample(images1, min(5, len(images1)))
        random_images2 = random.sample(images2, min(5, len(images2)))
        
        def display_images(images, title):
            plt.figure(figsize=(15, 3))
            for i, img_path in enumerate(images):
                try:
                    img = Image.open(img_path)
                    plt.subplot(1, 5, i + 1)
                    plt.imshow(img)
                    plt.axis('off')
                    plt.title(f"Imagen {i + 1}")
                except Exception as e:
                    print(f"Error cargando imagen {img_path}: {e}")
            plt.suptitle(title)
            plt.tight_layout()
            plt.show()
        
        display_images(random_images1, "5 imágenes aleatorias de la carpeta 1")
        display_images(random_images2, "5 imágenes aleatorias de la carpeta 2")
    else:
        print("Carpetas de imágenes no encontradas. Saltando visualización de imágenes.")

# Ejecutar la función
load_and_display_sample_images()


In [ ]:
def load_metadata():
    """Cargar y mostrar metadatos si están disponibles"""
    try:
        tabular_data = pd.read_csv('./skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
        print("Metadatos cargados exitosamente:")
        print(tabular_data.head())
        print(f"\nInformación del conjunto de datos:")
        print(f"Forma: {tabular_data.shape}")
        print(f"Columnas: {tabular_data.columns.tolist()}")
        return tabular_data
    except FileNotFoundError:
        print("Archivo de metadatos no encontrado. Continuando sin metadatos.")
        return None

# Cargar metadatos
metadata = load_metadata()

In [ ]:
def load_preprocessed_data():
    """Cargar los datos preprocesados del CSV"""
    try:
        data = pd.read_csv('./skin-cancer-mnist-ham10000/hmnist_28_28_RGB.csv')
        print(f"Datos cargados exitosamente. Forma: {data.shape}")
        print(f"Columnas: {data.columns.tolist()}")
        print(f"Distribución de etiquetas:")
        print(data['label'].value_counts().sort_index())
        return data
    except FileNotFoundError:
        print("Error: Archivo CSV no encontrado. Por favor verifica la ruta del archivo.")
        return None

# Cargar datos principales
data = load_preprocessed_data()

In [ ]:
# Preparar características y etiquetas
X = data.drop('label', axis=1).values
y = data['label'].values

print(f"Forma de datos originales: {X.shape}")
print(f"Forma de etiquetas: {y.shape}")
print(f"Clases: {np.unique(y)}")
print(f"Número de clases: {len(np.unique(y))}")

In [ ]:
def advanced_balance_data(x, y, method='oversample', random_state=SEED):
    """
    Advanced data balancing with multiple strategies
    """
    if hasattr(x, 'values'):
        x_array = x.values
    else:
        x_array = np.array(x)
    
    if hasattr(y, 'values'):
        y_array = y.values
    else:
        y_array = np.array(y)
    
    unique_classes, counts = np.unique(y_array, return_counts=True)
    print(f"Distribución de clases original: {dict(zip(unique_classes, counts))}")
    
    if method == 'oversample':
        # Sobremuestreo de clases minoritarias
        max_count = max(counts)
        x_balanced = []
        y_balanced = []
        
        for class_label in unique_classes:
            class_indices = np.where(y_array == class_label)[0]
            class_x = x_array[class_indices]
            class_y = y_array[class_indices]
            
            if len(class_x) < max_count:
                class_x_resampled, class_y_resampled = resample(
                    class_x, class_y, 
                    n_samples=max_count, 
                    random_state=random_state
                )
            else:
                class_x_resampled, class_y_resampled = class_x, class_y
                
            x_balanced.append(class_x_resampled)
            y_balanced.append(class_y_resampled)
        
        x_final = np.vstack(x_balanced)
        y_final = np.hstack(y_balanced)
        
        print(f"Distribución de clases balanceada: {dict(zip(*np.unique(y_final, return_counts=True)))}")
        return x_final, y_final

# Aplicar balanceo de datos
X_balanced, y_balanced = advanced_balance_data(X, y, method='oversample')


In [ ]:

# Reshape and normalize
X_balanced = X_balanced.reshape(-1, 28, 28, 3).astype(np.float32)

# Advanced normalization
mean = np.mean(X_balanced, axis=(0, 1, 2), keepdims=True)
std = np.std(X_balanced, axis=(0, 1, 2), keepdims=True)
X_balanced = (X_balanced - mean) / (std + 1e-8)

print(f"Balanced data shape: {X_balanced.shape}")
print(f"Data range after normalization: [{X_balanced.min():.3f}, {X_balanced.max():.3f}]")
print(f"Mean: {X_balanced.mean():.3f}, Std: {X_balanced.std():.3f}")

In [ ]:
def create_data_augmentation_layer():
    """Create a data augmentation layer using tf.keras"""
    return tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomBrightness(0.1),
        tf.keras.layers.RandomContrast(0.1),
    ])

def advanced_augment_images(images, labels, augment_factor=0.3):
    """Advanced data augmentation with multiple techniques"""
    num_augmented = int(augment_factor * len(images))
    rng = np.random.default_rng(SEED)
    indices = rng.choice(len(images), num_augmented, replace=False)
    
    x_selected = images[indices]
    y_selected = labels[indices]
    
    # Create augmentation pipeline
    augmentation_layer = create_data_augmentation_layer()
    
    # Apply augmentation
    x_augmented = []
    for img in x_selected:
        img_expanded = tf.expand_dims(img, 0)
        aug_img = augmentation_layer(img_expanded, training=True)
        
        # Add noise
        noise = tf.random.normal(shape=tf.shape(aug_img), mean=0.0, stddev=0.01)
        aug_img = aug_img + noise
        
        # Clip values
        aug_img = tf.clip_by_value(aug_img, -3.0, 3.0)
        x_augmented.append(tf.squeeze(aug_img, 0))
    
    x_augmented = tf.stack(x_augmented)
    return x_augmented.numpy(), y_selected

# Apply data augmentation
print("Applying data augmentation...")
X_aug, y_aug = advanced_augment_images(X_balanced, y_balanced, augment_factor=0.2)

# Combine original and augmented data
X_final = np.concatenate([X_balanced, X_aug], axis=0)
y_final = np.concatenate([y_balanced, y_aug], axis=0)

print(f"Final dataset shape: {X_final.shape}")
print(f"Original data: {X_balanced.shape[0]} samples")
print(f"Augmented data: {X_aug.shape[0]} samples")
print(f"Total data: {X_final.shape[0]} samples")


In [ ]:
# Split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    X_final, y_final, test_size=0.3, random_state=SEED, stratify=y_final
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=SEED, stratify=y_temp
)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

# Check class distribution in each set
print(f"\nTraining set class distribution: {dict(zip(*np.unique(y_train, return_counts=True)))}")
print(f"Validation set class distribution: {dict(zip(*np.unique(y_val, return_counts=True)))}")
print(f"Test set class distribution: {dict(zip(*np.unique(y_test, return_counts=True)))}")

In [ ]:
def create_improved_lenet(input_shape=(28, 28, 3), num_classes=7, dropout_rate=0.3):
    """
    Improved LeNet-5 with modern techniques
    """
    model = Sequential([
        # First convolutional block
        Conv2D(32, (5, 5), activation='relu', input_shape=input_shape, 
               kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate * 0.5),
        
        # Second convolutional block
        Conv2D(64, (5, 5), activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate * 0.5),
        
        # Third convolutional block (additional)
        Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(dropout_rate * 0.5),
        
        # Global Average Pooling instead of Flatten
        GlobalAveragePooling2D(),
        
        # Dense layers
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(dropout_rate),
        
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(dropout_rate),
        
        # Output layer
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Create the improved LeNet model
model_improved = create_improved_lenet()
model_improved.summary()


In [ ]:
def create_lightweight_cnn(input_shape=(28, 28, 3), num_classes=7, dropout_rate=0.2):
    """
    Lightweight CNN for comparison
    """
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Create lightweight CNN model
model_lightweight = create_lightweight_cnn()
model_lightweight.summary()

In [ ]:
def create_deep_cnn(input_shape=(28, 28, 3), num_classes=7, dropout_rate=0.4):
    """
    Deeper CNN with residual-like connections
    """
    model = Sequential([
        # Block 1
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate * 0.5),
        
        # Block 2
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate * 0.5),
        
        # Block 3
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate * 0.5),
        
        # Classification head
        GlobalAveragePooling2D(),
        Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Create deep CNN model
model_deep = create_deep_cnn()
model_deep.summary()

In [ ]:
def create_advanced_callbacks(model_name):
    """Create modern callbacks for training"""
    callbacks = [
        # Model checkpoint
        ModelCheckpoint(
            filepath=f'best_{model_name}.keras',
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        ),
        
        # Early stopping
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Learning rate reduction
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        )
    ]
    
    return callbacks

def cosine_decay_with_warmup(epoch, total_epochs=50, warmup_epochs=5, 
                           initial_lr=0.001, min_lr=1e-6):
    """
    Cosine decay learning rate schedule with warmup
    """
    if epoch < warmup_epochs:
        # Linear warmup
        lr = initial_lr * (epoch + 1) / warmup_epochs
    else:
        # Cosine decay
        progress = (epoch - warmup_epochs) / (total_epochs - warmup_epochs)
        lr = min_lr + (initial_lr - min_lr) * 0.5 * (1 + np.cos(np.pi * progress))
    
    return lr

# Test the learning rate schedule
epochs_test = np.arange(30)
lrs_test = [cosine_decay_with_warmup(epoch, total_epochs=30) for epoch in epochs_test]

plt.figure(figsize=(10, 4))
plt.plot(epochs_test, lrs_test, linewidth=2)
plt.xlabel('Epochs')
plt.ylabel('Learning Rate')
plt.title('Cosine Decay Learning Rate Schedule with Warmup')
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

In [ ]:
# Compile improved LeNet model
model_improved.compile(
    optimizer=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Create callbacks
callbacks_improved = create_advanced_callbacks("improved_lenet")

# Add learning rate scheduler
lr_scheduler = LearningRateScheduler(
    lambda epoch: cosine_decay_with_warmup(epoch, total_epochs=30)
)
callbacks_improved.append(lr_scheduler)

print("Training Improved LeNet...")
print(f"Model parameters: {model_improved.count_params():,}")

start_time = time.time()

# Train the improved model
history_improved = model_improved.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=64,
    epochs=30,
    callbacks=callbacks_improved,
    verbose=1
)

training_time_improved = time.time() - start_time
print(f"Training completed in {training_time_improved:.2f} seconds")

In [ ]:
model_lightweight.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Create callbacks
callbacks_lightweight = create_advanced_callbacks("lightweight_cnn")
callbacks_lightweight.append(
    LearningRateScheduler(lambda epoch: cosine_decay_with_warmup(epoch, total_epochs=30))
)

print("Training Lightweight CNN...")
print(f"Model parameters: {model_lightweight.count_params():,}")

start_time = time.time()

# Train the lightweight model
history_lightweight = model_lightweight.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=64,
    epochs=30,
    callbacks=callbacks_lightweight,
    verbose=1
)

training_time_lightweight = time.time() - start_time
print(f"Training completed in {training_time_lightweight:.2f} seconds")

In [ ]:
# Compile deep model
model_deep.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Create callbacks
callbacks_deep = create_advanced_callbacks("deep_cnn")
callbacks_deep.append(
    LearningRateScheduler(lambda epoch: cosine_decay_with_warmup(epoch, total_epochs=30))
)

print("Training Deep CNN...")
print(f"Model parameters: {model_deep.count_params():,}")

start_time = time.time()

# Train the deep model
history_deep = model_deep.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=64,
    epochs=30,
    callbacks=callbacks_deep,
    verbose=1
)

training_time_deep = time.time() - start_time
print(f"Training completed in {training_time_deep:.2f} seconds")

In [ ]:
def comprehensive_evaluation(model, X_test, Y_test, X_train, Y_train, 
                           history, model_name, class_names=None):
    """
    Comprehensive model evaluation with multiple metrics
    """
    print(f"\n{'='*50}")
    print(f"EVALUATION RESULTS FOR {model_name.upper()}")
    print(f"{'='*50}")
    
    # Evaluate on train and test sets
    train_loss, train_acc = model.evaluate(X_train, Y_train, verbose=0)
    test_loss, test_acc = model.evaluate(X_test, Y_test, verbose=0)
    
    print(f"Training Accuracy: {train_acc:.4f}")
    print(f"Training Loss: {train_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Generalization Gap: {train_acc - test_acc:.4f}")
    
    # Predictions for confusion matrix
    y_pred = model.predict(X_test, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    # Classification report
    if class_names is None:
        class_names = [f'Class {i}' for i in range(7)]
    
    print("\nDetailed Classification Report:")
    print(classification_report(Y_test, y_pred_classes, target_names=class_names))
    
    return {
        'train_accuracy': train_acc,
        'test_accuracy': test_acc,
        'train_loss': train_loss,
        'test_loss': test_loss,
        'generalization_gap': train_acc - test_acc,
        'predictions': y_pred_classes
    }

# Define class names
class_names = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']

In [ ]:
# Evaluate improved LeNet
results_improved = comprehensive_evaluation(
    model_improved, X_test, y_test, X_train, y_train, 
    history_improved, "Improved LeNet", class_names
)

results_improved['training_time'] = training_time_improved

In [ ]:
# Evaluate lightweight CNN
results_lightweight = comprehensive_evaluation(
    model_lightweight, X_test, y_test, X_train, y_train, 
    history_lightweight, "Lightweight CNN", class_names
)

results_lightweight['training_time'] = training_time_lightweight

In [ ]:
# Evaluate deep CNN
results_deep = comprehensive_evaluation(
    model_deep, X_test, y_test, X_train, y_train, 
    history_deep, "Deep CNN", class_names
)

results_deep['training_time'] = training_time_deep

In [ ]:
# Plot training history for improved LeNet
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Improved LeNet - Training Results and Analysis', fontsize=16)

# Training history
axes[0, 0].plot(history_improved.history['loss'], label='Training Loss', linewidth=2)
axes[0, 0].plot(history_improved.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0, 0].set_xlabel('Epochs')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training and Validation Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(history_improved.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[0, 1].plot(history_improved.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0, 1].set_xlabel('Epochs')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].set_title('Training and Validation Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Confusion Matrix
cm = confusion_matrix(y_test, results_improved['predictions'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1, 0],
            xticklabels=class_names, yticklabels=class_names)
axes[1, 0].set_xlabel('Predicted Label')
axes[1, 0].set_ylabel('True Label')
axes[1, 0].set_title('Confusion Matrix')

# Learning rate schedule
if 'lr' in history_improved.history:
    axes[1, 1].plot(history_improved.history['lr'], linewidth=2, color='red')
    axes[1, 1].set_xlabel('Epochs')
    axes[1, 1].set_ylabel('Learning Rate')
    axes[1, 1].set_title('Learning Rate Schedule')
    axes[1, 1].set_yscale('log')
    axes[1, 1].grid(True, alpha=0.3)
else:
    # Class distribution
    unique, counts = np.unique(y_test, return_counts=True)
    axes[1, 1].bar(range(len(class_names)), counts, color='skyblue', alpha=0.7)
    axes[1, 1].set_xlabel('Classes')
    axes[1, 1].set_ylabel('Count')
    axes[1, 1].set_title('Test Set Class Distribution')
    axes[1, 1].set_xticks(range(len(class_names)))
    axes[1, 1].set_xticklabels(class_names, rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Plot training history for lightweight CNN
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Lightweight CNN - Training Results and Analysis', fontsize=16)

# Training history
axes[0, 0].plot(history_lightweight.history['loss'], label='Training Loss', linewidth=2)
axes[0, 0].plot(history_lightweight.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0, 0].set_xlabel('Epochs')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training and Validation Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(history_lightweight.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[0, 1].plot(history_lightweight.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0, 1].set_xlabel('Epochs')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].set_title('Training and Validation Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Confusion Matrix
cm = confusion_matrix(y_test, results_lightweight['predictions'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', ax=axes[1, 0],
            xticklabels=class_names, yticklabels=class_names)
axes[1, 0].set_xlabel('Predicted Label')
axes[1, 0].set_ylabel('True Label')
axes[1, 0].set_title('Confusion Matrix')

# Model parameters comparison
models_params = [
    model_improved.count_params(),
    model_lightweight.count_params(),
    model_deep.count_params()
]
model_names_short = ['Improved\nLeNet', 'Lightweight\nCNN', 'Deep\nCNN']
axes[1, 1].bar(model_names_short, models_params, color=['blue', 'green', 'red'], alpha=0.7)
axes[1, 1].set_xlabel('Models')
axes[1, 1].set_ylabel('Parameters')
axes[1, 1].set_title('Model Parameters Comparison')
for i, v in enumerate(models_params):
    axes[1, 1].text(i, v + max(models_params) * 0.01, f'{v:,}', ha='center', va='bottom')

plt.tight_layout()
plt.show()